# 查詢自由時報關鍵字，儲存至MongoDB

In [1]:
#coding:utf-8
#65001
import urllib.request
import json
import codecs
import sys
import argparse as ap
import time
import datetime
import lxml.html
import pymongo as mg
from urllib.parse import quote

#python main.py 世大運 2015-07-01 2017-07-03 1
# def argParse():
#     parser=ap.ArgumentParser(description='Liberty Time Net Crawler')
#     parser.add_argument("keyword", help="Serch Keyword")
#     parser.add_argument("start_date", help="Start (2017-01-01)")
#     parser.add_argument("end_date", help="End (2017-01-02)")
#     parser.add_argument("pages", help="Pages")
#     return parser.parse_args()

# args=argParse()
# keyword = quote(args.keyword)
# start_date = args.start_date
# end_date = args.end_date
# pages = args.pages

keyword = quote('川普')
start_date = '2018-07-01'
end_date = '2018-07-19'
pages = '1'

client = mg.MongoClient('127.0.0.1:27017')
db = client['ltnews']

def start_requests():
    start_list = start_date.split("-")
    end_list = end_date.split("-")
    SYear = ''
    SMonth = ''
    SDay = ''
    EYear = ''
    EMonth = ''
    EDay = ''
    if(len(start_list)==3) and (len(end_list)==3):
        SYear = start_list[0]
        SMonth = start_list[1]
        SDay = start_list[2]
        EYear = end_list[0]
        EMonth = end_list[1]
        EDay = end_list[2]
    else:
        print ("Date format error.")
  
    urls = []
    for i in range(1,int(pages)+1):
        str_idx = ''+('%s' % i)
        api = 'http://news.ltn.com.tw/search?keyword='+keyword+'&conditions=and&SYear='+SYear+'&SMonth='+SMonth+'&SDay='+SDay+'&EYear='+EYear+'&EMonth='+EMonth+'&EDay='+EDay+'&page='+str_idx+''
        urls.append(api)
        for url in urls:
            #print (url)
            parseLtnNews(url)
            time.sleep(0.2)


def parseLtnNews(uri):
    handle = urllib.request.urlopen(uri)
    encoding = handle.headers.get_content_charset()
    html_data =  handle.read().decode(encoding)
    selector = lxml.html.document_fromstring(html_data)
    newslist = selector.xpath('//*[@class="searchlist boxTitle"]/li')
    
    for i in range(len(newslist)):
        strTitle = ''
        strUrl = ''
        strBody = ''
        strDate = ''
        str_idx = str(i+1)
        str_xpath = '//*[@class="searchlist boxTitle"]/li['+str_idx+']/a//text()'
        titleList = selector.xpath(str_xpath)
        strTitle = " ".join(titleList)
        print(strTitle)
        str_xpath = '//*[@class="searchlist boxTitle"]/li['+str_idx+']/a//@href'
        urlList = selector.xpath(str_xpath)[0]
        strUrl = ''.join(urlList)
        strUrl = strUrl
        print(strUrl)
        str_xpath = '//*[@class="searchlist boxTitle"]/li['+str_idx+']/p//text()'
        bodyList = selector.xpath(str_xpath)
        strBody = ''.join(bodyList).replace('\n','')
        str_xpath = '//*[@class="searchlist boxTitle"]/li['+str_idx+']/span//text()'
        dateList = selector.xpath(str_xpath)
        strDate = ''.join(dateList).replace("&nbsp;","")[:10]
        if len(strTitle)>1:
            result = db.urllib.insert_one({
                "title": strTitle,
                "link":strUrl,
                "body":strBody,
                "postdate":strDate,
                "datetime":datetime.datetime.now(),
                "updatetime":datetime.datetime.now().strftime('%Y-%m-%d')
            })
    handle.close()



if __name__ == '__main__':
    items = []
    start_requests();
    row_json = json.dumps(items, ensure_ascii=False)
    file = codecs.open(urllib.parse.unquote(keyword)+'.json', 'w', encoding='utf-8')
    #file = codecs.open('out.json', 'w', encoding='utf-8')
    file.write(row_json)
    file.close()
    #print(row_json)
    print("Done")

15
薇拉夫人的國際關係料理藝術》 川普 與普欽高峰會：美俄關係的舊問題與新挑戰
俄國阿嬤外表激似 川普  網路爆紅
歐盟下週與美協商汽車關稅  川普 ：結果不公將遭「巨大報復」
川普 新關稅？ 美國對進口鈾啟「232調查」
日企憂貿易戰 6月對美出口17個月來首跌
川普 挺俄被罵賣國賊 FBI局長表態：俄國確實介入美國大選
前白宮戰略顧問放話 美國會打贏貿易戰！
金融公司齊轟 川普 貿易戰 庫德洛：要就怪中國和歐洲
大叔男星自豪集滿體位 遇20歲鮮肉演員大解放
不怕貿易戰？ 微信支付計劃擴大美國市場
美國防部「有權派航母通過台海」 我重申支持公海暢行
蕭美琴等立委在美關切台灣鋼鋁被課關稅
貿易戰干擾 工研院下修我國製造業今年產值
看到鬼？ 川普 妻與普廷握手後 嚇到表情變這樣...
「虧雞福來爹」預算噤口 黃光芹驚問林義豐自我定位
Done
